In [1]:
import pandas as pd
import gzip
import json
import seaborn as sns
import numpy as np
import time
import html
import re
import os
from typing import *
import urllib.request
from PIL import Image
import cv2 as cv
from matplotlib import pyplot as plt
from tqdm import tqdm, trange
tqdm.pandas()

In [2]:
%%time
metadata_test = pd.read_csv('../../metadata_parsed_with_images.csv')

Wall time: 10.3 s


In [3]:
metadata_test = metadata_test.iloc[0:300000]

In [4]:
metadata_test.shape

(300000, 12)

In [5]:
metadata_test = metadata_test.fillna('')

In [6]:
import ast

In [7]:
%%time
metadata_test['images'] = metadata_test['images'].apply(lambda x: ast.literal_eval(x))

Wall time: 2.39 s


In [8]:
%%time
metadata_test['imagesHighRes'] = metadata_test['imagesHighRes'].apply(lambda x: ast.literal_eval(x))

Wall time: 2.33 s


In [9]:
part = metadata_test.copy()

In [10]:
import traceback
from functools import partial

In [11]:
import psutil, humanize

def mem_report():
    print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))

mem_report()

CPU RAM Free: 10.5 GB


In [12]:
import mapply

In [13]:
part.head()

,rank,category,description,title,brand,feature,price,asin,images,imagesHighRes,category_id,rank_scaled
0,11120.0,"Arts, Crafts & Sewing > Craft Supplies > Soap ...",,Pinkie Tm girl flower Handmade soap silicone m...,,,35.00,7121277158,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,1,1.953892
1,2574.0,"Arts, Crafts & Sewing > Craft Supplies > Soap ...",,Pinkie Tm 3D blooming chrysanthemums Flower So...,,,35.00,7121280027,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,1,2.589388
2,761776.0,Kitchen & Dining (See Top 100 in Kitchen & Din...,moldsize:L7.2xW3.7xH4.8cm(L2.83xW1.46xH1.89inc...,Pinkie Tm Rabbit animal silicone soap mold for...,pinkie,"Made in 100% pure silicone,softness and comfor...",7.99,7121281821,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,2,0.118169
3,636.0,"Arts, Crafts & Sewing > Craft Supplies > Ceram...",moldsize:L7.2xW3.7xH4.8cm(L2.83xW1.46xH1.89inc...,Pinkie Tm Rabbit animal silicone soap mold for...,pinkie,"Made in 100% pure silicone,softness and comfor...",7.99,7121281821,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,3,3.196539
4,7303.0,"Arts, Crafts & Sewing > Craft Supplies > Soap ...",,Pinkie Tm Womens Wallets handbag purse shaped ...,,,35.00,7121282534,[],[],1,2.136495


In [14]:
metadata_test.shape

(300000, 12)

In [15]:
import traceback

def image_url(urls, path='images_0/'):
    import traceback
    try:
        if len(urls) == 0:
            return ''
        url = urls[0]
        return path + url[49:]
    except Exception:
        print(f'{traceback.format_exc()}')
        return ''

In [16]:
metadata_test['image_count'] = metadata_test.images.apply(lambda x: len(x))

In [17]:
parts = []

for i in trange(3):
    l = i * 100000
    r = min(l + 100000, metadata_test.shape[0])
    print(f"[{l}:{r}]")
    mem_report()
    part = metadata_test.iloc[l:r]
    print('images')
    part['image_url'] = part.images.apply(partial(image_url, path=f'images_{i}/'))
    print('images high res')
    part['image_url_high_res'] = part.imagesHighRes.apply(partial(image_url, path=f'images_high_res_{i}/'))
    parts.append(part)
    
df = pd.concat(parts)

  0%|          | 0/3 [00:00<?, ?it/s]C:\Users\artyo\AppData\Local\Temp/ipykernel_3320/2156869551.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part['image_url'] = part.images.apply(partial(image_url, path=f'images_{i}/'))
C:\Users\artyo\AppData\Local\Temp/ipykernel_3320/2156869551.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part['image_url_high_res'] = part.imagesHighRes.apply(partial(image_url, path=f'images_high_res_{i}/'))
 67%|██████▋   | 2/3 [00:00<00:00, 11.26it/s]

[0:100000]
CPU RAM Free: 10.5 GB
images
images high res
[100000:200000]
CPU RAM Free: 10.4 GB
images
images high res
[200000:300000]
CPU RAM Free: 10.4 GB
images


100%|██████████| 3/3 [00:00<00:00, 10.42it/s]

images high res


In [18]:
df[df['category_id'] == 4].shape

(97618, 15)

In [19]:
df = df.sort_values(by='category_id')

In [20]:
df['category_rank'] = df.groupby(['category_id']).cumcount()

In [21]:
df['category_id_batched'] = df.category_rank.apply(lambda x: x // 40)

In [22]:
df['category_id'] = (df['category_id'] + 1) * 10000 + df['category_id_batched'] 

In [23]:
len(set(df.category_id))

7694

In [24]:
metadata_parsed = df.copy()

In [25]:
metadata_parsed['category'].iloc[0]

'Home & Kitchen (See Top 100 in Home & Kitchen)'

In [26]:
def remove_seetop100(x):
    start = x.lower().find('see top 100') 
    if start != -1:
        return x[:start - 1]
    return x

In [27]:
metadata_parsed['category'] = metadata_parsed['category'].apply(lambda x: remove_seetop100(x))

In [28]:
metadata_parsed

,rank,category,description,title,brand,feature,price,asin,images,imagesHighRes,category_id,rank_scaled,image_count,image_url,image_url_high_res,category_rank,category_id_batched
126217,653496.0,Home & Kitchen,"This lightweight, 1 oz, natural goatskin rawhi...","LeatherUnltd Rawhide Goatskin, 1 oz, 7-8 sq ft",LeatherUnltd,"1 oz natural goatskin rawhide, Ideal for Lace,...",49.00,B00B1G5LJC,[],[],10000,0.184754,0,,,0,0
91611,250169.0,Home & Kitchen,", Excellent for Applications where Accurate Sp...",Master Airbrush Model G44 High Precision Detai...,Master Airbrush,"""Masters best selling detail airbrush thats lo...",1.09,B006CVSPJ4,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,10000,0.601763,6,images_0/411PKunVn7L._SS40_.jpg,images_high_res_0/411PKunVn7L.jpg,1,0
91625,87207.0,Home & Kitchen,Fold out melamine table with 1 in. gridded sur...,Sullivans Quilters Table,Sullivans,Fold out melamine table with 1 in. gridded sur...,240.99,B006CX19X6,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,10000,1.059445,5,images_0/41iYnhdU-wL._SS40_.jpg,images_high_res_0/41iYnhdU-wL.jpg,2,0
91637,274037.0,Home & Kitchen,", Pack includes two sheets of top quality Mill...",Mill Hill Perforated Paper - Antique Brown - 9...,Mill Hill,Package includes two sheets of top quality per...,10.25,B006CYU4ZY,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,10000,0.562187,1,images_0/61evD-HCAZL._SS40_.jpg,images_high_res_0/61evD-HCAZL.jpg,3,0
91653,750470.0,Home & Kitchen,Yarn Content: 50% Microfiber Acrylic 50% Nylon...,Dreambaby D.K. Yarn 100 White,Plymouth Yarn,,9.95,B006D06S48,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,10000,0.124663,8,images_0/41ouquTN1qL._SS40_.jpg,images_high_res_0/41ouquTN1qL.jpg,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241328,73454.0,Electronics > Home Audio & Theater,"""Finally, a wool-free sock yarn! Knitters and ...",HiKoo CoBaSi Multi (814 Wave Caps),HiKoo,"Yarn: Fingering Weight Sock yarn, 55% Cotton y...",8.95,B00SHSXX7O,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,7790000,1.133981,7,images_2/51vXaWnGEWL._SS40_.jpg,images_high_res_2/51vXaWnGEWL.jpg,0,0
250873,492290.0,Electronics,Quick Overview \nThis 127*30cm 3D carbon fiber...,EverTrust(TM) 127*30CM 3D Carbon Fiber Film Vi...,EverTrust,"100% brand new and high quality, 127*30cm 3D c...",12.38,B00UV57VA6,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,7860000,0.307776,4,images_2/412nZ%2BNk43L._SS36_.jpg,images_high_res_2/412nZ%2BNk43L.jpg,0,0
253359,6231.0,Computers & Accessories,Size: 6mm Color: Black and White Self-adhesive...,DECORA 6mm Round Wiggly Toy Eye with Self-adhe...,DECORA,1000pcs 6mm googly eyes in a resealable zip ba...,5.99,B00VG51N1I,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,7890000,2.205439,6,images_2/41qHDxQqi%2BL._SS40_.jpg,images_high_res_2/41qHDxQqi%2BL.jpg,0,0
253357,8736.0,Computers & Accessories,Size: 12mm Color: Black and White Self-adhes...,DECORA 1000 Pieces 12mm Round Wiggle Googly Ey...,DECORA,"Black/White Wiggle Eyes With Self-adhesive, Th...",8.99,B00VG51WY6,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,7890000,2.058684,6,images_2/41qHDxQqi%2BL._SS40_.jpg,images_high_res_2/41qHDxQqi%2BL.jpg,1,0


### Посчитаем NDCG рандомного предсказания

Для начала нужно понять, что такое рандомное предсказание. По-хорошему BERT будет предсказывать новые ранги, а не просто сортировать уже имеющиеся. В качестве довольно плохого приближения можно исплользовать равномерное распределение на [0, 6]
А есть нормальная идея: взять весь список рангов и выбирать из него рандомные

In [29]:
category_ids = set(list(metadata_parsed.category_id))

In [30]:
all_ranks = list(metadata_parsed["rank_scaled"])

In [31]:
print(len(all_ranks))

300000


In [32]:
import random

NUM_ITEMS = 40

real_ranks = []
random_ranks = []

for category in category_ids:
    a = list(metadata_parsed[metadata_parsed['category_id'] == category].sort_values(by='rank').iloc[0:NUM_ITEMS]['rank_scaled'])
    if len(a) < 40:
        continue
    real_ranks.append(a)
    random_ranks.append(random.sample(all_ranks, NUM_ITEMS))

In [33]:
print(np.array(real_ranks).shape)
print(np.array(random_ranks).shape)

(7317, 40)
(7317, 40)


In [34]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_ranking as tfr

In [35]:
from tqdm import trange

In [91]:
ndcg = tfr.keras.metrics.NDCGMetric(topn=5)
ngcgs = []
for i in trange(len(real_ranks)):
    ngcgs.append(ndcg(real_ranks[i:i+1], random_ranks[i:i+1]).numpy())

100%|██████████| 7317/7317 [01:49<00:00, 66.88it/s]


In [92]:
np.array(ngcgs).mean()

0.37005424

In [62]:
for i in range(1, 41):
    ndcg = tfr.keras.metrics.NDCGMetric(topn=i)
    print(f"NDCG@{i}: {ndcg(real_ranks, random_ranks).numpy()}")

NDCG@1: 0.27741721272468567
NDCG@2: 0.303310364484787
NDCG@3: 0.32507145404815674
NDCG@4: 0.3434201180934906
NDCG@5: 0.3600502014160156
NDCG@6: 0.3761254549026489
NDCG@7: 0.39083534479141235
NDCG@8: 0.4047471880912781
NDCG@9: 0.4179899990558624
NDCG@10: 0.43005138635635376
NDCG@11: 0.44192999601364136
NDCG@12: 0.4535923898220062
NDCG@13: 0.4647643268108368
NDCG@14: 0.4757555425167084
NDCG@15: 0.48630157113075256
NDCG@16: 0.4966924786567688
NDCG@17: 0.5067606568336487
NDCG@18: 0.5164785385131836
NDCG@19: 0.5263077020645142
NDCG@20: 0.5360348224639893
NDCG@21: 0.5454726219177246
NDCG@22: 0.5549251437187195
NDCG@23: 0.5642771124839783
NDCG@24: 0.5732648968696594
NDCG@25: 0.5825837850570679
NDCG@26: 0.5915178060531616
NDCG@27: 0.60047447681427
NDCG@28: 0.6096016764640808
NDCG@29: 0.6185185313224792
NDCG@30: 0.6272718906402588
NDCG@31: 0.636314332485199
NDCG@32: 0.6452332735061646
NDCG@33: 0.6542215943336487
NDCG@34: 0.663281261920929
NDCG@35: 0.6723036170005798
NDCG@36: 0.6814430356025696


Подмешиваем какой-то процент рандомных рангов в настоящие. При этом хорошо бы настоящие ранги заменить на другие, но тоже отсортированные 

### Хочу нормально насчитать NDCG@1, NDCG@3, NDCG@5 и NDCG@10 для 30% рандомных рангов, но кажется это мем

In [36]:
real_ranks = np.array(pd.DataFrame(real_ranks).sample(frac=1))

In [37]:
random_ranks = np.array(pd.DataFrame(random_ranks).sample(frac=1))

In [38]:
print(np.array(real_ranks).shape)
print(np.array(random_ranks).shape)

(7317, 40)
(7317, 40)


In [39]:
real_ranks

array([[3.18574893, 2.86264241, 2.50375898, ..., 1.63079923, 1.6082677 ,
        1.60791541],
       [3.51712294, 3.3362956 , 3.01772529, ..., 0.97485458, 0.61518092,
        0.61044064],
       [3.15242387, 2.70575081, 1.56184989, ..., 0.0307456 , 0.02940653,
        0.01873076],
       ...,
       [2.22973379, 1.89873074, 1.7070541 , ..., 0.05658264, 0.04191642,
        0.03452436],
       [3.59345635, 3.08883892, 2.88239383, ..., 0.61486055, 0.56934108,
        0.56034021],
       [2.00537147, 1.98421777, 1.96169435, ..., 0.59928761, 0.56080125,
        0.55533197]])

In [40]:
import random
percentage = [0., 10., 20., 25., 30., 50., 100.]

for p in percentage:
    target_ranks = []
    predicted_ranks = []
    for i, (real, rand) in enumerate(zip(real_ranks, random_ranks)): 
        for i in range(10):  
            nice = [] 
            for g, b in zip(real_ranks[(i + 1) // len(real_ranks)], rand):
                n = random.random()    
                if n >= p / 100.:
                    nice.append(g)
                else:
                    nice.append(b) 
            predicted_ranks.append(nice)
            target_ranks.append(real)

    ndcg = tfr.keras.metrics.NDCGMetric()
    print(f"Percentage: {p}")
    for i in [1, 3, 5, 10, 40]:
        ndcg = tfr.keras.metrics.NDCGMetric(topn=i)
        print(f"NDCG@{i}: {ndcg(target_ranks, predicted_ranks).numpy()}")

Percentage: 0.0
NDCG@1: 1.0
NDCG@3: 1.0
NDCG@5: 1.0
NDCG@10: 1.0
NDCG@40: 1.0
Percentage: 10.0
NDCG@1: 0.7496519684791565
NDCG@3: 0.834526002407074
NDCG@5: 0.8668624758720398
NDCG@10: 0.8979381322860718
NDCG@40: 0.9390802383422852
Percentage: 20.0
NDCG@1: 0.5768909454345703
NDCG@3: 0.7048552632331848
NDCG@5: 0.7554416060447693
NDCG@10: 0.8117515444755554
NDCG@40: 0.8898542523384094
Percentage: 25.0
NDCG@1: 0.5121918320655823
NDCG@3: 0.649610161781311
NDCG@5: 0.7060869932174683
NDCG@10: 0.772880494594574
NDCG@40: 0.8685073256492615
Percentage: 30.0
NDCG@1: 0.4576380252838135
NDCG@3: 0.5997496247291565
NDCG@5: 0.6610116362571716
NDCG@10: 0.7365870475769043
NDCG@40: 0.8490076661109924
Percentage: 50.0
NDCG@1: 0.3358527719974518
NDCG@3: 0.45553356409072876
NDCG@5: 0.5236450433731079
NDCG@10: 0.6170800924301147
NDCG@40: 0.7898462414741516
Percentage: 100.0
NDCG@1: 0.2747800052165985
NDCG@3: 0.3228565752506256
NDCG@5: 0.35969698429107666
NDCG@10: 0.4283210337162018
NDCG@40: 0.718034625053405

In [40]:
print(np.array(target_ranks).shape)
print(np.array(predicted_ranks).shape)

(11810, 40)
(11810, 40)


Получается, что я стремлюсь всего лишь к 
Percentage: 30.0  
NDCG@3: 0.627774715423584  
NDCG@5: 0.677333652973175  
NDCG@10: 0.7556678056716919  
NDCG@20: 0.8400372266769409  
NDCG@40: 0.8945969343185425  

Сейчас на уровне 50%, что конечно нехорошо, но и не ужасно

In [46]:
from sklearn.model_selection import GroupShuffleSplit

In [47]:
gss = GroupShuffleSplit(test_size=.33, n_splits=1, random_state = 7).split(metadata_parsed, groups=metadata_parsed['category_id'])

X_train_inds, X_test_inds = next(gss)

train_data = metadata_parsed.iloc[X_train_inds]
test_data = metadata_parsed.iloc[X_test_inds]

In [48]:
train_data.to_csv('train_with_images.csv', index=False)

In [49]:
test_data.to_csv('test_with_images.csv', index=False)

In [50]:
test_data = pd.read_csv('test_with_images.csv')

In [51]:
test_data.shape

(99166, 17)

In [52]:
len(set(test_data.category_id))

2540

In [53]:
eval_categories = []

for i in set(test_data.category_id):
    l = test_data[test_data['category_id'] == i].sort_values(by='rank').sample(frac=1)
    assert l.shape[0] <= 40
    eval_categories.append(l)    
eval_data = pd.concat(eval_categories)

In [54]:
len(set(eval_data.category_id))

2540

In [55]:
eval_data.shape

(99166, 17)

In [56]:
eval_data.to_csv('eval_with_images.csv', index=False)

In [57]:
eval_data.head()

,rank,category,description,title,brand,feature,price,asin,images,imagesHighRes,category_id,rank_scaled,image_count,image_url,image_url_high_res,category_rank,category_id_batched
71977,219.0,"Arts, Crafts & Sewing > Printmaking > Etching ...",This steel ball burnisher is ideal for flatten...,Steel Tip Ball Burnisher with Wood Handle,FineArtStore,NaN,9.99,B001KWAXLI,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,1450001,3.659552,1,images_0/41b4xDcKyyL._SS40_.jpg,images_high_res_0/41b4xDcKyyL.jpg,64,1
71967,87.0,"Arts, Crafts & Sewing > Printmaking > Etching ...",Steel Double Ended Spiral Twist Etching Needle...,Steel Twisted Double Pointed Etching Needle (7...,FineArtStore,"Solid steel double ended etching needle, Spira...",10.99,B001KM4Z8A,[],[],1450001,4.060477,0,NaN,NaN,54,1
71976,453.0,"Arts, Crafts & Sewing > Printmaking > Etching ...",Glass Etching Starter Kit- .ARMOUR PRODUCTS-St...,Armour Products - Glass Etching Starter Kit-,Armour,"Armour Products - Glass Etching Starter Kit-, ...",23.06,B00I3AYF8M,[],[],1450001,3.343898,0,NaN,NaN,63,1
71956,177.0,"Arts, Crafts & Sewing > Printmaking > Etching ...",Get the Versatility of a Diamond Engraver with...,Armour Diamond Point Glass Engraving Set,Delphi Glass,Get the versatility of an engraver with the co...,2.65,B000YQMS14,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,1450001,3.752023,3,images_0/31UmAqL7XxL._SS40_.jpg,images_high_res_0/31UmAqL7XxL.jpg,43,1
71981,95.0,"Arts, Crafts & Sewing > Printmaking > Etching ...",You are buying 8 Ounces (by weight) of Nickel ...,Nickel Sulfate NiSO4 99.9% ACS Grade Crystals ...,Eisen-Golden Laboratories,NaN,3.00,B00WUDS14M,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,1450001,4.022273,4,images_2/41mthIHpq8L._SS40_.jpg,images_high_res_2/41mthIHpq8L.jpg,68,1
